# Step 1: instantiate

In [6]:
!pip3 install praw
!pip3 install pyvis
!pip3 install python-decouple
!pip3 install dotenv
import praw
from prawcore.exceptions import Forbidden
import numpy as np
import pandas as pd
from pyvis.network import Network
import networkx as nx
import matplotlib.pyplot as plt
from dotenv import load_dotenv
load_dotenv()
import os

client_id = os.getenv("6MlGd7qJwditcQhFdfv0IA")
secret = os.getenv("_YN9ZFYhpvpsOAVl3Ou-jU6eF80tYg")

reddit = praw.Reddit(
    client_secret="_YN9ZFYhpvpsOAVl3Ou-jU6eF80tYg",
    client_id="6MlGd7qJwditcQhFdfv0IA",
    user_agent="http://localhost"
)

print(reddit.user.me())

None


# Step 2: Define functions

## Step 2.1: define function to obtain recent redditors of a subreddit

## Step 2.2: define fucntion to obtain recent subreddits to which a redditor has posted to

In [8]:
def get_users(sub="schizoid", post_count=8, comment_count=12):
    """
    Finds the most recently active users in a subreddit. It's easier to comment than to post on a subreddit, so more comments are collected.
    :param subreddit: subreddit to crawl
    :param posts_count: number of posts to search for
    :param comments_count: number of comments to search for
    :return: List of usernames who were recently active in a subreddit
    """
    # === Obtain list of posters and commentors ===
    posters = []

    try:
        for s in reddit.subreddit(sub).new(limit=post_count):
            if s.author.name != "AutoModerator":
                posters.append(s.author.name)
            elif type(s.author.name) == None:
                pass
    except Forbidden:
        print("Tried to access Forbidden")
        #pass

    """posters = [
        s.author.name                                         #<- Redditor who posted
        for s in reddit.subreddit(sub).new(limit=post_count)  #<- from most recent posts
        if s.author.name != "AutoModerator"                   #<- remove auto moderator
    ]"""


    commentors = []

    try:
        # --- Same with commentors ---
        for c in reddit.subreddit(sub).comments(limit=comment_count):
            if c.author.name != "AutoModerator":
                commentors.append(c.author.name)
            elif type(c.author.name) == None:
                pass
    except Forbidden:
        print("Tried to access Forbidden")
        #pass
    """
    commenters = [
        c.author.name
        for c in reddit.subreddit(sub).comments(limit=comment_count)
        if c.author.name != "AutoModerator"
    ]"""

    # === Combine the posters and commentors ===
    redditors = posters + commentors
    # --- Remove duplicates
    unique_redditors = list(set(redditors))

    return unique_redditors

In [9]:
def get_redditor_activity(subreddit, post_count=8, comment_count=12, filter_soi=True):
    """
    Get names of subreddits to which users have posted to.
    :param redditor_list: list of redditor usernames
    :param lim: number of posts to limit to; default 12
    """

    # === Define lists to store subreddits where redditor has posted and commented to ===
    posts_subreddits = []
    comments_subreddits = []

    # ===
    # Iterate over list of redditors and store the names of the subreddits where they posted and store
    # them in the list to a certain length
    # ===

    redditor_list = get_users(subreddit, post_count, comment_count)

    # -- Iterate over list of redditors --
    for r in redditor_list:

        posts = reddit.redditor(r).submissions.new(limit=post_count) #<- new submissions from redditors
        comments = reddit.redditor(r).comments.new(limit=comment_count)

        try:
            # - Iterate over posts/submissions -
            for submission in posts:
                # Skip if they posted in the subreddit that we're scraping, otherwise store to list
                if submission.subreddit.display_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        posts_subreddits.append(
                            {
                                "source": subreddit,
                                "target": submission.subreddit.display_name,
                                "type": "submission",
                                'weight': 3
                            }
                        )

                elif submission.subreddit.display_name != sub_of_interest:
                    posts_subreddits.append(
                        {
                            "source": subreddit,
                            "target": submission.subreddit.display_name,
                            "type": "submission",
                            'weight': 3
                        }
                    )

            for comment in comments:
                if comment.subreddit.display_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        comments_subreddits.append(
                                {
                                    "source": subreddit,
                                    "target": comment.subreddit.display_name,
                                    "type": "comment",
                                    'weight': 1
                                }
                        )
                elif comment.subreddit.display_name != sub_of_interest:
                        comments_subreddits.append(
                            {
                                "source": subreddit,
                                "target": comment.subreddit.display_name,
                                "type": "comment",
                                'weight': 1
                            }
                        )
        except Forbidden:
            print("Forbidden")
            pass

    #activity_list = posts_subreddits + comments_subreddits

    return posts_subreddits + comments_subreddits


def remove_self_loops(dataframe, column_1, column_2):

    mask = dataframe.apply(lambda x: x[column_1] in x[column_2], axis=1)

    df = dataframe[~mask]

    return df

# Step 3: get redditors from the sub of interest and their recently active subreddits

In [12]:
primary_activity = get_redditor_activity("schizoid", post_count=7, comment_count=10, filter_soi=True)

print(primary_activity[:5])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



NameError: name 'sub_of_interest' is not defined

In [16]:
def get_redditor_activity(subreddit, post_count=8, comment_count=12, filter_soi=True): # Change schizoid to subreddit
    """
    Get names of subreddits to which users have posted to.
    :param redditor_list: list of redditor usernames
    :param lim: number of posts to limit to; default 12
    """

    # === Define sub_of_interest here ===
    sub_of_interest = subreddit # Change schizoid to subreddit

    # === Define lists to store subreddits where redditor has posted and commented to ===
    posts_subreddits = []
    comments_subreddits = []

    # ===
    # Iterate over list of redditors and store the names of the subreddits where they posted and store
    # them in the list to a certain length
    # ===

    redditor_list = get_users(subreddit, post_count, comment_count)

    # -- Iterate over list of redditors --
    for r in redditor_list:

        posts = reddit.redditor(r).submissions.new(limit=post_count) #<- new submissions from redditors
        comments = reddit.redditor(r).comments.new(limit=comment_count)

        try:
            # - Iterate over posts/submissions -
            for submission in posts:
                # Skip if they posted in the subreddit that we're scraping, otherwise store to list
                if submission.subreddit.display_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        posts_subreddits.append(
                            {
                                "source": subreddit,
                                "target": submission.subreddit.display_name,
                                "type": "submission",
                                'weight': 3
                            }
                        )

                elif submission.subreddit.display_name != sub_of_interest:
                    posts_subreddits.append(
                        {
                            "source": subreddit,
                            "target": submission.subreddit.display_name,
                            "type": "submission",
                            'weight': 3
                        }
                    )

            for comment in comments:
                if comment.subreddit.display_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        comments_subreddits.append(
                                {
                                    "source": subreddit,
                                    "target": comment.subreddit.display_name,
                                    "type": "comment",
                                    'weight': 1
                                }
                        )
                elif comment.subreddit.display_name != sub_of_interest:
                        comments_subreddits.append(
                            {
                                "source": subreddit,
                                "target": comment.subreddit.display_name,
                                "type": "comment",
                                'weight': 1
                            }
                        )
        except Forbidden:
            print("Forbidden")
            pass

    #activity_list = posts_subreddits + comments_subreddits

    return posts_subreddits + comments_subreddits

In [17]:
primary_activity = get_redditor_activity("schizoid", post_count=7, comment_count=10, filter_soi=True)

# The following line assigns the result of get_redditor_activity to primary_activity
# print(primary_activity[:5])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

# Step 4: Organize the edge list into a dataframe

In [64]:
df_primary = pd.DataFrame(
    data=primary_activity
)

df_primary = remove_self_loops(df_primary, "source", "target")

display(df_primary)
print(df_primary.shape)

,source,target,type,weight
0,schizoid,Schizoid,submission,3
1,schizoid,lowendgaming,submission,3
2,schizoid,SchizoidAdjacent,submission,3
3,schizoid,Schizoid,submission,3
4,schizoid,Schizoid,submission,3
...,...,...,...,...
225,schizoid,portoalegre,comment,1
226,schizoid,portoalegre,comment,1
227,schizoid,C_Programming,comment,1
228,schizoid,Engenharia,comment,1


(230, 4)


# Step 5: iterate over the primary networks to get secondary activity edge list

In [24]:
def get_redditor_activity(subreddit, post_count=8, comment_count=12, filter_soi=True):
    """
    Get names of subreddits to which users have posted to.
    :param redditor_list: list of redditor usernames
    :param lim: number of posts to limit to; default 12
    """

    # === Define sub_of_interest here ===
    sub_of_interest = subreddit

    # === Define lists to store subreddits where redditor has posted and commented to ===
    posts_subreddits = []
    comments_subreddits = []

    # ===
    # Iterate over list of redditors and store the names of the subreddits where they posted and store
    # them in the list to a certain length
    # ===

    redditor_list = get_users(subreddit, post_count, comment_count)

    # -- Iterate over list of redditors --
    for r in redditor_list:

        posts = reddit.redditor(r).submissions.new(limit=post_count) #<- new submissions from redditors
        comments = reddit.redditor(r).comments.new(limit=comment_count)

        try:
            # - Iterate over posts/submissions -
            for submission in posts:
                # Skip if they posted in the subreddit that we're scraping, otherwise store to list
                # Check if subreddit exists before accessing it
                try:
                    subreddit_name = submission.subreddit.display_name
                except AttributeError:
                    print(f"Skipping invalid subreddit for user {r}")
                    continue  # Skip to the next submission

                if subreddit_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        posts_subreddits.append(
                            {
                                "source": subreddit,
                                "target": subreddit_name,
                                "type": "submission",
                                'weight': 3
                            }
                        )

                elif subreddit_name != sub_of_interest:
                    posts_subreddits.append(
                        {
                            "source": subreddit,
                            "target": subreddit_name,
                            "type": "submission",
                            'weight': 3
                        }
                    )

            for comment in comments:
                # Check if subreddit exists before accessing it
                try:
                    subreddit_name = comment.subreddit.display_name
                except AttributeError:
                    print(f"Skipping invalid subreddit for user {r}")
                    continue  # Skip to the next comment

                if subreddit_name == sub_of_interest:
                    if filter_soi is True:
                        pass
                    else:
                        comments_subreddits.append(
                                {
                                    "source": subreddit,
                                    "target": subreddit_name,
                                    "type": "comment",
                                    'weight': 1
                                }
                        )
                elif subreddit_name != sub_of_interest:
                        comments_subreddits.append(
                            {
                                "source": subreddit,
                                "target": subreddit_name,
                                "type": "comment",
                                'weight': 1
                            }
                        )
        except Forbidden:
            print("Forbidden")
            pass


    return posts_subreddits + comments_subreddits

In [65]:
def get_redditor_activity(subreddit, post_count=5, comment_count=8, filter_soi=True, depth=1):
    """
    Get names of subreddits to which users have posted to, with a depth limit.
    :param subreddit: The subreddit to analyze.
    :param post_count: Number of recent posts to consider per user (default 5).
    :param comment_count: Number of recent comments to consider per user (default 8).
    :param filter_soi: If True, filters out activity within the original subreddit (default True).
    :param depth: The depth of the analysis (1 for primary connections only, default 1).
    """

    sub_of_interest = subreddit
    posts_subreddits = []
    comments_subreddits = []

    redditor_list = get_users(subreddit, post_count, comment_count)

    for r in redditor_list:
        posts = reddit.redditor(r).submissions.new(limit=post_count)
        comments = reddit.redditor(r).comments.new(limit=comment_count)

        try:
            for submission in posts:
                try:
                    subreddit_name = submission.subreddit.display_name
                except AttributeError:
                    print(f"Skipping invalid subreddit for user {r}")
                    continue

                if depth == 1:  # Limit to primary connections only
                    if filter_soi and subreddit_name == sub_of_interest:
                        continue
                    posts_subreddits.append({
                        "source": subreddit,
                        "target": subreddit_name,
                        "type": "submission",
                        'weight': 3
                    })

            for comment in comments:
                try:
                    subreddit_name = comment.subreddit.display_name
                except AttributeError:
                    print(f"Skipping invalid subreddit for user {r}")
                    continue

                if depth == 1:  # Limit to primary connections only
                    if filter_soi and subreddit_name == sub_of_interest:
                        continue
                    comments_subreddits.append({
                        "source": subreddit,
                        "target": subreddit_name,
                        "type": "comment",
                        'weight': 1
                    })

        except Forbidden:
            print("Forbidden")
            pass

    return posts_subreddits + comments_subreddits

In [66]:
df_secondary = pd.DataFrame()

primary_target = list(df_primary.target.unique())

for subreddit in primary_target:
    secondary_activity = get_redditor_activity(subreddit, post_count=2, comment_count=5, filter_soi=False)

    # Use pd.concat instead of append
    df_secondary = pd.concat([df_secondary, pd.DataFrame(secondary_activity)], ignore_index=True)

# This line is redundant and can be removed
# df_secondary = pd.concat([df_secondary, pd.DataFrame(secondary_activity)], ignore_index=True)

display(df_secondary.head(25))

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

NotFound: received 404 HTTP response

# Step 6: concatenate the primary and secondary edge list into a single large dataframe

In [67]:
df = pd.concat([df_primary, df_secondary], ignore_index=True)

display(df.head())
display(df.tail())
print(df.shape)

,source,target,type,weight
0,schizoid,Schizoid,submission,3
1,schizoid,lowendgaming,submission,3
2,schizoid,SchizoidAdjacent,submission,3
3,schizoid,Schizoid,submission,3
4,schizoid,Schizoid,submission,3


,source,target,type,weight
1092,BPDlovedones,doener,comment,1
1093,BPDlovedones,doener,comment,1
1094,BPDlovedones,doener,comment,1
1095,BPDlovedones,BPDlovedones,comment,1
1096,BPDlovedones,FragReddit,comment,1


(1097, 4)


# Step 7: remove self-loops

In [68]:
df.to_csv(f"reddit_activity_schizoid.csv", mode='a', header=False, index=False)

# Step 8: combine the weights for each edge combination

In [69]:
d = {'weight':'sum', 'type': 'first'}
df = df.groupby(
    ['source','target'],
    sort=False,
    as_index=False
).agg(d).reindex(columns=df.columns)

display(df.head(25))

,source,target,type,weight
0,schizoid,Schizoid,submission,144
1,schizoid,lowendgaming,submission,10
2,schizoid,SchizoidAdjacent,submission,22
3,schizoid,AskWomen,submission,3
4,schizoid,librandu,submission,3
5,schizoid,EldenRingLoreTalk,submission,9
6,schizoid,Helldivers,submission,3
7,schizoid,AgingParents,submission,3
8,schizoid,Survival,submission,3
9,schizoid,ask,submission,3


# Step 7: define nodes and edges

In [70]:
s = df.source.unique().tolist()
t = df.target.unique().tolist()
n = list(set(s+t))

nodes = {v: k for k, v in enumerate(n, 0)}

df['source_num'] = df['source'].map(nodes)
df['target_num'] = df['target'].map(nodes)

display(df.head(30))
#edge_list = [(s, t, w) for s, t, w in zip()]

,source,target,type,weight,source_num,target_num
0,schizoid,Schizoid,submission,144,10,321
1,schizoid,lowendgaming,submission,10,10,26
2,schizoid,SchizoidAdjacent,submission,22,10,234
3,schizoid,AskWomen,submission,3,10,67
4,schizoid,librandu,submission,3,10,404
5,schizoid,EldenRingLoreTalk,submission,9,10,130
6,schizoid,Helldivers,submission,3,10,124
7,schizoid,AgingParents,submission,3,10,356
8,schizoid,Survival,submission,3,10,103
9,schizoid,ask,submission,3,10,259


In [71]:
node_list = list(nodes.values())
node_labels = list(nodes.keys())
edge_list = [(source, target, weight) for source, target, weight in zip(df.source_num, df.target_num, df.weight)]

In [72]:
nx_pv = Network("500px", "1000px", notebook=True)

nx_pv.add_nodes(node_list, label=node_labels)

nx_pv.add_edges(edge_list)
nx_pv.show_buttons(filter_=['physics'])
nx_pv.save_graph(f"reddit_map_schizoid.html")

In [73]:
G = nx.DiGraph()
edge_list = [(source, target, {"weight": weight}) for source, target, weight in zip(df.source_num, df.target_num, df.weight)]

print(edge_list[:5])

G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

[(10, 321, {'weight': 144}), (10, 26, {'weight': 10}), (10, 234, {'weight': 22}), (10, 67, {'weight': 3}), (10, 404, {'weight': 3})]


In [74]:
!pip install --upgrade pyvis
from IPython.display import IFrame
from pyvis.network import Network

# ... (Your previous code to create the graph G) ...

# Create a Pyvis network object with cdn_resources='in_line'
nx_pv = Network("500px", "1000px", notebook=True, cdn_resources='in_line')

# Add nodes and edges to the Pyvis network
nx_pv.add_nodes(node_list, label=node_labels)  # Assuming you have node_list and node_labels
nx_pv.add_edges(edge_list)  # Assuming you have edge_list

# Customize the visualization (optional)
nx_pv.show_buttons(filter_=['physics'])  # Show buttons for interactive controls

# Display or save the graph
nx_pv.show(f"reddit_map_schizoid.html")
IFrame(src='./reddit_map_schizoid.html', width='100%', height='500px')

reddit_map_schizoid.html


In [62]:
!pip install --upgrade pyvis  # Ensure Pyvis is up-to-date
from pyvis.network import Network
import pandas as pd

# ... (Your existing code to get data into the 'df' DataFrame) ...

# Create the Pyvis network object
nx_pv = Network("500px", "1000px", notebook=True, cdn_resources='in_line')

# Assuming 'df' has columns 'source_num', 'target_num', 'weight', 'source', 'target'
# Add nodes to the network
node_list = list(set(df['source_num'].tolist() + df['target_num'].tolist()))
node_labels = list(set(df['source'].tolist() + df['target'].tolist()))
nx_pv.add_nodes(node_list, label=node_labels)

# Add edges to the network
edge_list = [(source, target, weight) for source, target, weight
             in zip(df.source_num, df.target_num, df.weight)]
nx_pv.add_edges(edge_list)

# Save the graph to an HTML file
nx_pv.show("reddit_map_schizoid.html")
print("Graph saved as reddit_map_schizoid.html. You can now download it.")

KeyError: 'source_num'

In [63]:
!pip install --upgrade pyvis  # Ensure Pyvis is up-to-date
from pyvis.network import Network
import pandas as pd

# ... (Your existing code to get data into the 'df' DataFrame) ...

# Create the Pyvis network object
nx_pv = Network("500px", "1000px", notebook=True, cdn_resources='in_line')

# Assuming 'df' has columns 'source', 'target', and 'weight'
# 1. Recreate nodes dictionary
s = df.source.unique().tolist()
t = df.target.unique().tolist()
n = list(set(s+t))

nodes = {v: k for k, v in enumerate(n, 0)}

# 2. Recreate 'source_num' and 'target_num' columns
df['source_num'] = df['source'].map(nodes)
df['target_num'] = df['target'].map(nodes)

# Add nodes to the network
node_list = list(set(df['source_num'].tolist() + df['target_num'].tolist()))
node_labels = list(set(df['source'].tolist() + df['target'].tolist()))
nx_pv.add_nodes(node_list, label=node_labels)

# Add edges to the network
edge_list = [(source, target, weight) for source, target, weight
             in zip(df.source_num, df.target_num, df.weight)]
nx_pv.add_edges(edge_list)

# Save the graph to an HTML file
nx_pv.show("reddit_map_schizoid.html")
print("Graph saved as reddit_map_schizoid.html. You can now download it.")

reddit_map_schizoid.html
Graph saved as reddit_map_schizoid.html. You can now download it.
